In [46]:
import os

import skimage.feature
import skimage.transform.hough_transform as ht
import matplotlib.pyplot as plt
import numpy as np
import ipywidgets as widgets
from skimage.io import imread

### Load images

In [47]:
folder_1 = "images_vol_1"
folder_2 = "images_vol_2"
images = []

def load_images_from_folder(folder):
    images = []
    for filename in os.listdir(folder):
        if filename.endswith(".jpg") or filename.endswith(".png"):
            img = imread(os.path.join(folder, filename))
            if img is not None:
                scaled_image = skimage.transform.resize(img, (img.shape[0] // 2, img.shape[1] // 2))
                img_bw = skimage.color.rgb2gray(scaled_image)
                images.append(img_bw)
    return images

def crop_image(img, border_size=10):
    return img[border_size:-border_size, border_size:-border_size]

In [48]:
images = load_images_from_folder(folder_1)

In [49]:
@widgets.interact(image_index=(0, len(images) - 1, 1))
def plot_images(image_index):
    plt.imshow(images[image_index], cmap="gray")

interactive(children=(IntSlider(value=10, description='image_index', max=20), Output()), _dom_classes=('widget…

## Edge detection

In [90]:
def gaussian_filter(images, sigma=2.5):
    ims_filtered = []
    for img in images:
        ims_filtered.append(skimage.filters.gaussian(img, sigma=sigma, mode='constant', cval=0.0))
    return ims_filtered


def edge_detection(images):
    imgs_edges = []
    low_threshold = 0.1
    high_threshold = 0.3

    for img in images:
        imgs_edges.append(
            skimage.feature.canny(img, low_threshold=low_threshold, high_threshold=high_threshold))
    return imgs_edges

In [91]:
imgs_filtered = gaussian_filter(images)

In [92]:
@widgets.interact(image_index=(0, len(images) - 1, 1))
def plot_images(image_index):
    plt.imshow(imgs_filtered[image_index], cmap="gray")

interactive(children=(IntSlider(value=10, description='image_index', max=20), Output()), _dom_classes=('widget…

In [93]:
imgs_edges = edge_detection(imgs_filtered)

In [94]:
imgs_filtered = [crop_image(img) for img in imgs_filtered]
imgs_edges = [crop_image(img) for img in imgs_edges]

In [95]:
@widgets.interact(image_index=(0, len(images) - 1, 1))
def plot_lines(image_index):
    plt.imshow(imgs_edges[image_index])

interactive(children=(IntSlider(value=10, description='image_index', max=20), Output()), _dom_classes=('widget…

## Hough transform

In [113]:
def find_lines(imgs_edges, threshold=15):
    lines_list = []
    for img_edges in imgs_edges:
        lines = ht.probabilistic_hough_line(img_edges, threshold=threshold, line_length=150, line_gap=20)
        # select 4 longest lines
        # ht.hough_line(img_edges), threshold=threshold, dists=[20, 20], 
        line_peaks = ht.hough_line_peaks(lines, dists=(20, 20), angles= )
        lines_list.append(lines)
        lines_list.append(lines)
    return lines_list

imgs_lines = find_lines(imgs_edges, 100)

AttributeError: 'list' object has no attribute 'shape'

In [115]:
angles=np.linspace(-np.pi / 2, np.pi / 2, 180)
angles

array([-1.57079633, -1.55324553, -1.53569473, -1.51814394, -1.50059314,
       -1.48304234, -1.46549155, -1.44794075, -1.43038995, -1.41283915,
       -1.39528836, -1.37773756, -1.36018676, -1.34263597, -1.32508517,
       -1.30753437, -1.28998358, -1.27243278, -1.25488198, -1.23733118,
       -1.21978039, -1.20222959, -1.18467879, -1.167128  , -1.1495772 ,
       -1.1320264 , -1.11447561, -1.09692481, -1.07937401, -1.06182322,
       -1.04427242, -1.02672162, -1.00917082, -0.99162003, -0.97406923,
       -0.95651843, -0.93896764, -0.92141684, -0.90386604, -0.88631525,
       -0.86876445, -0.85121365, -0.83366286, -0.81611206, -0.79856126,
       -0.78101046, -0.76345967, -0.74590887, -0.72835807, -0.71080728,
       -0.69325648, -0.67570568, -0.65815489, -0.64060409, -0.62305329,
       -0.60550249, -0.5879517 , -0.5704009 , -0.5528501 , -0.53529931,
       -0.51774851, -0.50019771, -0.48264692, -0.46509612, -0.44754532,
       -0.42999453, -0.41244373, -0.39489293, -0.37734213, -0.35

In [97]:
print(f'On average {np.mean([len(lines) for lines in imgs_lines]):.1f} lines per image')

On average 3.6 lines per image


In [99]:
@widgets.interact(image_index=(0, len(images) - 1, 1), line_index=(0, len(imgs_lines[10]) - 1, 1), all_lines=True)
def plot_lines(image_index, line_index, all_lines):
    plt.imshow(imgs_filtered[image_index], cmap="gray")
    line = imgs_lines[image_index][line_index]
    print(f'Number of lines: {len(imgs_lines[image_index])}')
    if all_lines:
        for line in imgs_lines[image_index]:
            p0, p1 = line
            plt.plot((p0[0], p1[0]), (p0[1], p1[1]), color="red")
    else:
        p0, p1 = line
        plt.plot((p0[0], p1[0]), (p0[1], p1[1]), color="red")

interactive(children=(IntSlider(value=10, description='image_index', max=20), IntSlider(value=1, description='…